In [277]:
# if error < tol: # MSE in yhat and y is less than some threshold
#     break

import random
import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from numpy.linalg import inv # inverse matrix

boston = load_boston()
X = boston.data
y = boston.target
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

intercept = np.ones((X_train.shape[0],1))
X_train = np.concatenate((intercept, X_train), axis=1) # axis 1 is COLUMN

intercept = np.ones((X_test.shape[0],1))
X_test = np.concatenate((intercept, X_test), axis=1)

In [278]:
'''
def closed_form(X, y):
    return inv(X.T @ X) @ X.T @ y

theta = closed_form(X_train, y_train)
# print(theta)
yhat = X_test @ theta # multiply testing dataset with weights (yhat = predicted y)
print("X_test.shape =", X_test.shape, "theta.shape =", theta.shape, "yhat.shape =", yhat.shape)
# print(y_test.shape)
# print(yhat[56:60])
# print(y_test[56:60])

mse = ((y_test - yhat)**2).sum()/X_test.shape[0]
print(mse) # aka LOSS
'''

'\ndef closed_form(X, y):\n    return inv(X.T @ X) @ X.T @ y\n\ntheta = closed_form(X_train, y_train)\n# print(theta)\nyhat = X_test @ theta # multiply testing dataset with weights (yhat = predicted y)\nprint("X_test.shape =", X_test.shape, "theta.shape =", theta.shape, "yhat.shape =", yhat.shape)\n# print(y_test.shape)\n# print(yhat[56:60])\n# print(y_test[56:60])\n\nmse = ((y_test - yhat)**2).sum()/X_test.shape[0]\nprint(mse) # aka LOSS\n'

In [331]:
class LinearRegression:
    def __init__(self, method, alpha=0.0001, max_iter=1000, previous_loss=9999, threshold=0.0001, mini_batch=20):
        self.alpha = alpha
        self.max_iter = max_iter
        self.previous_loss = previous_loss
        self.threshold = threshold
        self.method = method
        self.mini_batch = mini_batch
    
    def gradient_descent(self, X_train, y_train):
        self.theta = np.zeros(X_train.shape[1])
        
        if self.method == "batch":
            for i in range(self.max_iter):
                self.yhat = h_theta(X_train, self.theta)
                loss = mse(self.yhat, y_train)
                loss_difference = self.previous_loss - loss
                if loss_difference < self.threshold:
                    iter_stop = i
                    print("ITERATION STOPPED AT", iter_stop)
                    break
                self.previous_loss = loss
                self.error = self.yhat - y_train
                self.grad = gradient(X_train, self.error)
                self.theta = self.theta - self.alpha*self.grad  
            print("MSE = ",loss)
            
        if self.method == "sto":
            for i in range(self.max_iter):
                random_sample = random.randint(0, (len(X_train)-1))
                X_train_sample = X_train[random_sample].reshape(1,-1)
                self.yhat = h_theta(X_train_sample, self.theta)
                self.error = self.yhat - y_train[random_sample]
                #print(X_train_sample.shape, "AND", self.error.shape)
                self.grad = gradient(X_train_sample, self.error)
                self.theta = self.theta - self.alpha*self.grad
            loss = mse(self.yhat, y_train[random_sample])
            print("MSE = ",loss)
            
        if self.method == "mini":
            count = 0 
            for i in range(self.max_iter):
                X_train_sample = X_train[count:count+self.mini_batch]
                y_train_sample = y_train[count:count+self.mini_batch]
                count += self.mini_batch
                if count >= len(X_train):
                    count = 0
                self.yhat = h_theta(X_train_sample, self.theta)
                self.error = self.yhat - y_train_sample
                #print(X_train_sample.shape, "AND", self.error.shape)
                self.grad = gradient(X_train_sample, self.error)
                self.theta = self.theta - self.alpha*self.grad
            loss = mse(self.yhat, y_train_sample)
            print("MSE = ",loss)
            
        #self.yhat = h_theta(X_test, self.theta)
        #self.mse = mse(self.yhat, y_test)

def h_theta(X_train, theta):
    return X_train @ theta

def mse(yhat, y_test):
    return ((yhat - y_test)**2).sum() / yhat.shape[0]

def gradient(X_train, error):
    return error @ X_train 

In [338]:
boston = LinearRegression(method="batch", max_iter = 1000, threshold=0.01)
boston.gradient_descent(X_train, y_train)

ITERATION STOPPED AT 137
MSE =  24.59144676113063


In [339]:
boston_sto = LinearRegression(method="sto", max_iter=9999)
boston_sto.gradient_descent(X_train, y_train)
# I think this is working but MSE can be very high, even with 9999 iterations ???

MSE =  392.48275441057757


In [351]:
boston_mini = LinearRegression(method="mini", max_iter=137, mini_batch=354)
boston_mini.gradient_descent(X_train, y_train)
# USING THE SAME BATCH SIZE AS X_train.shape[0] gives the same MSE as batch gradient descent from above! IT WORKS!!!

MSE =  24.601436999956984
